Load and setup necessary packages / includes

In [1]:
%pip install pandas numpy matplotlib lxml geopy folium sklearn

from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

import pandas as pd
import numpy as np
import requests

Note: you may need to restart the kernel to use updated packages.


## Prior Code

In [2]:
html_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
html_df = html_df[~html_df["Borough"].str.contains("Not assigned")]
html_df.loc[html_df['Neighbourhood']=='Not assigned', 'Neighbourhood'] = html_df['Borough']
# html_df = html_df.groupby('Postcode').agg({'Borough':'first', 
#                                  'Neighbourhood': ', '.join
#                                 }).reset_index()
lnglat_df = pd.read_csv('./Geospatial_Coordinates.csv')
lnglat_df = lnglat_df.rename(index=str, columns={"Postal Code": "Postcode"})
df = pd.merge(html_df, lnglat_df, on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


Map the dataframe

In [3]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

import folium
toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto)  
    
toronto

Strip all but Boroughs that container Toronto and Map it

In [4]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [5]:
toronto_comp = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_comp)  
    
toronto_comp

In [6]:
CLIENT_ID = 'YWK55AL4TC4YHZQBOJYM3CLUGK50CN1EVXZTFKPSUUTXQ5JG'
CLIENT_SECRET = 'JBPS0GVULIAEOX4QPS4KTPD33OZFQEG0MQGLGHSJAFOZ5FFW'
VERSION = '20180605'

In [7]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


'https://api.foursquare.com/v2/venues/explore?&client_id=YWK55AL4TC4YHZQBOJYM3CLUGK50CN1EVXZTFKPSUUTXQ5JG&client_secret=JBPS0GVULIAEOX4QPS4KTPD33OZFQEG0MQGLGHSJAFOZ5FFW&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

Get venues from each borough in Toronto dataframe

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

_*This can take a minute or two*_

In [9]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Lets group all the venues by Neighbourhood and count venues found

In [10]:
toronto_venues.groupby(['Neighbourhood']).agg({'Venue': 'count'})

,Venue
Neighbourhood,
Adelaide,100
Bathurst Quay,15
Berczy Park,57
Brockton,19
Business Reply Mail Processing Centre 969 Eastern,17
CN Tower,15
Cabbagetown,44
Central Bay Street,88
Chinatown,100


In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.00,0.00000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Bathurst Quay,0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
3,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
5,CN Tower,0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022727,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.00,0.00000,0.000000,0.00,0.011364,0.000000,0.000000,0.011364,0.000000,0.011364
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.01,0.01000,0.000000,0.00,0.060000,0.000000,0.030000,0.010000,0.000000,0.000000
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Top 10 venues for each Neighborhood in new Dataframe

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Gym,Bakery,Bar,Burger Joint
1,Bathurst Quay,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Steakhouse,Beer Bar,Bakery
3,Brockton,Breakfast Spot,Café,Coffee Shop,Grocery Store,Climbing Gym,Burrito Place,Stadium,Bar,Restaurant,Caribbean Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio


_k_-means to cluster into 5 clusters

In [15]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# # run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# # check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_data.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Brewery,Italian Restaurant
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Brewery,Italian Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Tea Room,Sporting Goods Shop,Fast Food Restaurant,Pizza Place
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Tea Room,Sporting Goods Shop,Fast Food Restaurant,Pizza Place
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Bakery,Italian Restaurant,Pizza Place,Gastropub,Breakfast Spot,Cosmetics Shop


In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [18]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,0,Coffee Shop,Park,Pub,Café,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Brewery,Italian Restaurant
1,Downtown Toronto,Regent Park,0,Coffee Shop,Park,Pub,Café,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Brewery,Italian Restaurant
2,Downtown Toronto,Ryerson,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Tea Room,Sporting Goods Shop,Fast Food Restaurant,Pizza Place
3,Downtown Toronto,Garden District,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Tea Room,Sporting Goods Shop,Fast Food Restaurant,Pizza Place
4,Downtown Toronto,St. James Town,0,Coffee Shop,Restaurant,Café,Hotel,Bakery,Italian Restaurant,Pizza Place,Gastropub,Breakfast Spot,Cosmetics Shop
5,East Toronto,The Beaches,0,Music Venue,Health Food Store,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
6,Downtown Toronto,Berczy Park,0,Coffee Shop,Cocktail Bar,Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Steakhouse,Beer Bar,Bakery
7,Downtown Toronto,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Chinese Restaurant,Middle Eastern Restaurant,Salad Place,Restaurant,Bubble Tea Shop,Bar
8,Downtown Toronto,Christie,0,Grocery Store,Café,Park,Nightclub,Diner,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Convenience Store
9,Downtown Toronto,Adelaide,0,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Gym,Bakery,Bar,Burger Joint


In [19]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Central Toronto,Roselawn,1,Garden,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [20]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,CN Tower,2,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
60,Downtown Toronto,Bathurst Quay,2,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
61,Downtown Toronto,Island airport,2,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
62,Downtown Toronto,Harbourfront West,2,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
63,Downtown Toronto,King and Spadina,2,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
64,Downtown Toronto,Railway Lands,2,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
65,Downtown Toronto,South Niagara,2,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court


In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Central Toronto,Moore Park,3,Playground,Tennis Court,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
50,Central Toronto,Summerhill East,3,Playground,Tennis Court,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
66,Downtown Toronto,Rosedale,3,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Central Toronto,Forest Hill North,4,Trail,Sushi Restaurant,Jewelry Store,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
35,Central Toronto,Forest Hill West,4,Trail,Sushi Restaurant,Jewelry Store,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
